In [1]:
import json
import pandas as pd
import plotly.express as px
import numpy as np
from dash import Dash,html,dcc,Input,Output
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc
import plotly.graph_objects as go

In [2]:
Covid_Data = pd.read_csv('Covid-Data.csv')
geojson = json.load(open('Covid-Provinces.geojson'))

In [3]:
district_lookup = {feature['properties']['Province']: feature
                       for feature in geojson['features']}

In [4]:
def get_highlights(selections, geojson=geojson, district_lookup=district_lookup):
        geojson_highlights = dict()
        for k in geojson.keys():
            if k != 'features':
                geojson_highlights[k] = geojson[k]
            else:
                geojson_highlights[k] = [district_lookup[selection] for selection in selections]        
        return geojson_highlights

In [5]:
def get_figure(selections, show_map):
        
        #Selections = What province is selected
        #show_map = what map should be shown , eg., 'Cases' or 'Deaths' ect...
        
        #------------------------------------# 
        #Creating figure 1, the Choropleth, indexed at [0]
        #------------------------------------# 
        # Base choropleth layer --------------#
        
        fig = px.choropleth_mapbox(Covid_Data, geojson=geojson, 
                                   color=show_map, 
                                   color_continuous_scale='portland',
                                   locations="Province", 
                                   featureidkey="properties.Province",
                                   opacity=0.5)
    
    
        # Second layer - Highlights ----------#
        if len(selections) > 0:
            # highlights contain the geojson information for only 
            # the selected districts
            highlights = get_highlights(selections)
    
            fig.add_trace(
                px.choropleth_mapbox(Covid_Data, geojson=highlights,
                                     color=show_map,
                                     locations="Province",
                                     featureidkey="properties.Province",                                 
                                     opacity=1).data[0])
            
        
        #------------------------------------#
        fig.update_layout(mapbox_style="carto-positron", 
                          mapbox_zoom=4,
                          mapbox_center={"lat": -30.454, "lon": 23.7073},
                          margin={"r":0,"t":0,"l":0,"b":0},
                          uirevision='constant')
                                  
        
        #------------------------------------# 
        #Creating figure 2, Pie chart, indexed at [1]
        #------------------------------------# 
        #Pie chart according to what type of map is selected
        pie = px.pie(Covid_Data, values=show_map, names='Province',title=f'Number of total {show_map}')
        
                
        #------------------------------------# 
        #Creating figure 3, Bar chart, indexed at [2]
        #------------------------------------# 
         #Bar graph , according to which ever province is selected
        selected_province = selections
        selected_df = (Covid_Data.loc[Covid_Data['Province'].isin(selected_province)])
        
        #Create bar graph
        bar = px.bar(selected_df, x='Province', y=['Cases','Deaths','Recoveries','Active Cases'],barmode='group',text_auto='.3s')
        bar.update_traces(textfont_size=13, textangle=0, textposition='outside',cliponaxis=False)
        #------------------------------------#
       
                
        return fig , pie , bar

In [6]:
#Set a default selected Province as 'selections' , in our default we select everything
selections = list(district_lookup.keys())[:]

#Set default data the choropleth should show as 'show_map', in our default we select 'Cases' to be shown
show_map = 'Cases'
figures = get_figure(selections, show_map)

#Initialize Correlation scatterplot
cor = px.scatter(Covid_Data, x='Cases', y='Deaths', trendline='ols')

In [7]:
# Keep track of the clicked region by using the variable \"selections\"
selections = set()


#initialize the app
app = JupyterDash(__name__,external_stylesheets=[dbc.themes.BOOTSTRAP])


app.layout = html.Div([
    dbc.Container([
        dbc.Row(dbc.Col(html.H2('An Interactive Choropleth Map of Covid in each Province of South-Africa')),justify='center'),
    
        html.Br(),
    
        dbc.Row([
        dbc.Col([html.H4('Choropleth of South-Africa')]),
        dbc.Col([html.H4('Data from Choropleth, represented in a Pie-Chart')])
        ]),

        dbc.Row([
            dbc.Col([
                dcc.Dropdown(['Cases','Deaths','Recoveries','Active Cases'],'Cases', id='Map_dd'),
                dcc.Graph(
                    id='choropleth',
                    figure=figures[0])
                ]),

            dbc.Col([
                dcc.Graph(
                    id='pie',
                    figure=figures[1])       
            ])
        ]),

        html.Br(),

        dbc.Row([
            dbc.Col(html.H4('Bar Graph of selected Provinces')),
            dbc.Col(html.H4('Correlation Graph between selected Variables')),
        ]),

        dbc.Row([
            dbc.Col([
                dcc.Graph(
                    id='Bar',
                    figure=figures[2])
            ],width=6),

            dbc.Col([
                    dcc.Dropdown(['Cases','Deaths','Recoveries','Active Cases'],'Cases',id='x_axis'),
                    dcc.Dropdown(['Cases','Deaths','Recoveries','Active Cases'],'Deaths',id='y_axis'),
                    dcc.Graph(id='cor',
                         figure=cor)
            ],width=6)
        ]),
        ],fluid=True)
])
#-------------------------------#
#Update our Correlation graph when values are changed
@app.callback(
    Output('cor','figure'),
    Input('x_axis','value'),
    Input('y_axis','value'))
def corr_graph(x_axis, y_axis):
        cor = px.scatter(Covid_Data, x=x_axis, y=y_axis, trendline='ols')
        return cor
    
    
@app.callback(
    Output('choropleth', 'figure'),
    Output('pie','figure'),
    Output('Bar','figure'),
    Input('Map_dd','value'),
    Input('choropleth', 'clickData'))
def update_figure(Map_dd, clickData): 
        
    #What data should the choropleth show, eg., 'Cases' or 'Deaths' ect...
    show_map = Map_dd
        
    if clickData is not None:            
        location = clickData['points'][0]['location']
           
        if location not in selections:
            selections.add(location)
        else:
            selections.remove(location)
        
    #Call funtion
    figures = get_figure(selections,show_map)
       
    return figures[0],figures[1],figures[2]
    
#-------------------------------# 

In [8]:
#Run the app.
app.run_server(mode='external')

Dash app running on http://127.0.0.1:8050/
